In [1]:
# 预测全家桶与机器学习神器

In [2]:
## Attrition

In [3]:
### Attrition Catboost

In [1]:
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')
path = os.path.expanduser('~/data/')

In [2]:
trainset=pd.read_csv(path + 'BI/train.csv',index_col=0)
testset=pd.read_csv(path + 'BI/test.csv',index_col=0)

In [3]:
#print(train['Attrition'].value_counts())
# 处理Attrition字段
trainset['Attrition']=trainset['Attrition'].map(lambda x:1 if x=='Yes' else 0)


In [4]:
from sklearn.preprocessing import LabelEncoder
# 查看数据是否有空值
#print(train.isna().sum())

In [5]:
# 去掉没用的列 员工号码，标准工时（=80）
train = trainset.drop(['EmployeeNumber', 'StandardHours'], axis=1)
test = testset.drop(['EmployeeNumber', 'StandardHours'], axis=1)

In [6]:
# 对于分类特征进行特征值编码
attr=['Age','BusinessTravel','Department','Education','EducationField','Gender','JobRole','MaritalStatus','Over18','OverTime']
lbe_list=[]
for feature in attr:
    lbe=LabelEncoder()
    train[feature]=lbe.fit_transform(train[feature])
    test[feature]=lbe.transform(test[feature])
    lbe_list.append(lbe)
#print(train)

In [7]:
import catboost as cb
import numpy as np
from sklearn.model_selection import train_test_split

ModuleNotFoundError: No module named 'catboost'

In [32]:
X_train, X_valid, y_train, y_valid = train_test_split(train.drop('Attrition',axis=1), train['Attrition'], test_size=0.2, random_state=42)

In [33]:
model = cb.CatBoostClassifier(iterations=1000, 
                              depth=7, 
                              learning_rate=0.01, 
                              loss_function='Logloss', 
                              eval_metric='AUC',
                              logging_level='Verbose', 
                              metric_period=50
                             )

In [34]:
# 得到分类特征的列号
categorical_features_indices = []
for i in range(len(X_train.columns)):
    if X_train.columns.values[i] in attr:
        categorical_features_indices.append(i)
print(categorical_features_indices)

[0, 1, 3, 5, 6, 9, 13, 15, 19, 20]


In [35]:
model.fit(X_train, y_train, eval_set=(X_valid, y_valid), cat_features=categorical_features_indices)

0:	test: 0.6390374	best: 0.6390374 (0)	total: 2.7ms	remaining: 2.69s
50:	test: 0.7893703	best: 0.7893703 (50)	total: 159ms	remaining: 2.95s
100:	test: 0.7950453	best: 0.7950453 (100)	total: 329ms	remaining: 2.93s
150:	test: 0.8011568	best: 0.8011568 (150)	total: 502ms	remaining: 2.82s
200:	test: 0.7958092	best: 0.8011568 (150)	total: 695ms	remaining: 2.76s
250:	test: 0.8023573	best: 0.8023573 (250)	total: 889ms	remaining: 2.65s
300:	test: 0.8034487	best: 0.8034487 (300)	total: 1.06s	remaining: 2.45s
350:	test: 0.8046491	best: 0.8046491 (350)	total: 1.24s	remaining: 2.29s
400:	test: 0.8030121	best: 0.8046491 (350)	total: 1.43s	remaining: 2.13s
450:	test: 0.8054131	best: 0.8054131 (450)	total: 1.64s	remaining: 1.99s
500:	test: 0.8067227	best: 0.8067227 (500)	total: 1.86s	remaining: 1.85s
550:	test: 0.8062862	best: 0.8067227 (500)	total: 2.04s	remaining: 1.66s
600:	test: 0.8056313	best: 0.8067227 (500)	total: 2.23s	remaining: 1.48s
650:	test: 0.8053039	best: 0.8067227 (500)	total: 2.42s	r

In [36]:
#model = cb.train(param, train_data, evals=[(train_data, 'train'), (valid_data, 'valid')], num_boost_round = 10000, early_stopping_rounds=200, verbose_eval=25)
predict = model.predict(test)
#predict = model.predict_proba(test)
#print(predict)
test['Attrition']=predict

In [37]:
## 转化为二分类输出
#test['Attrition']=test['Attrition'].map(lambda x:1 if x>=0.5 else 0)
# test[['Attrition']].to_csv('submit_cb.csv')

In [38]:
## Attrition GBDT

In [39]:
import pandas as pd

In [40]:
#print(train['Attrition'].value_counts())
# 处理Attrition字段
train['Attrition']=trainset['Attrition'].map(lambda x:1 if x=='Yes' else 0)

In [41]:
from sklearn.preprocessing import LabelEncoder
# 查看数据是否有空值
#print(train.isna().sum())

In [42]:
# 去掉没用的列 员工号码，标准工时（=80）
train = trainset.drop(['EmployeeNumber', 'StandardHours'], axis=1)
test = testset.drop(['EmployeeNumber', 'StandardHours'], axis=1)

In [43]:
# 对于分类特征进行特征值编码
attr=['Age','BusinessTravel','Department','Education','EducationField','Gender','JobRole','MaritalStatus','Over18','OverTime']
lbe_list=[]
for feature in attr:
    lbe=LabelEncoder()
    train[feature]=lbe.fit_transform(train[feature])
    test[feature]=lbe.transform(test[feature])
    lbe_list.append(lbe)
#train.to_csv('temp.csv')
#print(train)

In [44]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split

In [45]:
X_train, X_valid, y_train, y_valid = train_test_split(train.drop('Attrition',axis=1), train['Attrition'], test_size=0.2, random_state=42)

In [46]:
# 采用回归算法，可以得到更好的AUC结果
model = GradientBoostingRegressor(random_state=10)
model.fit(X_train, y_train)
predict = model.predict(test)
print(predict)
test['Attrition']=predict
#print(predict)

[ 1.01678366e-01  4.47445681e-02  1.42121556e-01  7.57204880e-02
  7.03200210e-01  3.70055651e-01  3.74376235e-01  4.37242139e-02
 -1.15634448e-01  2.97987772e-01  1.20779518e-01  1.09366950e-01
  1.07679394e-01  7.66600582e-01  7.51844882e-02 -4.76111832e-02
  4.28285053e-02 -1.09673242e-02  1.59486732e-01  9.62417811e-02
  6.47325348e-01  1.10040397e-01 -9.94554567e-03 -4.55439809e-02
  2.68815031e-01  3.37430996e-01  7.35105859e-02  1.06048931e-01
  5.50770434e-01  4.15464265e-02  1.79826162e-01  1.01207422e-03
  1.58722155e-01  1.58671176e-01  8.62998751e-02  2.67636183e-02
  1.24005756e-01  9.94003577e-02  8.73177129e-02  1.59141438e-03
  3.02853331e-02 -5.02317582e-03 -2.69498786e-03  8.75329962e-03
  3.69313030e-02  5.03418977e-01  2.62794122e-01 -1.16559089e-02
  8.28838190e-01  4.78927949e-01  2.30745577e-01  3.61855221e-01
  1.97193403e-01  2.08480315e-01  4.27257886e-01  7.31510619e-02
 -4.10414290e-02  1.41280282e-01 -5.16541193e-02  4.63709471e-01
  8.91047041e-03  1.57167

In [47]:
# 转化为二分类输出
#test['Attrition']=test['Attrition'].map(lambda x:1 if x>=0.5 else 0)
# test[['Attrition']].to_csv('submit_gbdt.csv')
print('submit_gbdt.csv saved')

submit_gbdt.csv saved


In [48]:
### Attrition LGB Onehot

In [12]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.feature_extraction import DictVectorizer

In [13]:
train=pd.read_csv(path + 'BI/train.csv',index_col=0)
test=pd.read_csv(path + 'BI/test.csv',index_col=0)
#print(train['Attrition'].value_counts())

In [14]:
# 处理Attrition字段
train['Attrition']=train['Attrition'].map(lambda x:1 if x=='Yes' else 0)

# 查看数据是否有空值
#print(train.isna().sum())

In [15]:
# 去掉没用的列 员工号码，标准工时（=80）
train = train.drop(['EmployeeNumber', 'StandardHours'], axis=1)
test = test.drop(['EmployeeNumber', 'StandardHours'], axis=1)

In [16]:
# 对于分类特征进行特征值编码
attr=['Age','BusinessTravel','Department','Education','EducationField','Gender','JobRole','MaritalStatus','Over18','OverTime']

In [17]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

In [18]:
param = {'boosting_type':'gbdt',
                         'objective' : 'binary:logistic', #
                         'eval_metric' : 'auc',
                         'eta' : 0.01,
                         'max_depth' : 15,
                         'colsample_bytree':0.8,
                         'subsample': 0.9,
                         'subsample_freq': 8,
                         'alpha': 0.6,
                         'lambda': 0,
        }

In [19]:
X_train, X_valid, y_train, y_valid = train_test_split(train.drop('Attrition',axis=1), train['Attrition'], test_size=0.2, random_state=42)

In [20]:
# 使用DictVectorizer对离散值进行编码
dvec = DictVectorizer(sparse=False)
X_train = dvec.fit_transform(X_train.to_dict(orient='record'))
X_valid = dvec.transform(X_valid.to_dict(orient='record'))
temp_test = dvec.transform(test.to_dict(orient='record'))

In [21]:
# 进行DMatrix转换
train_data = xgb.DMatrix(X_train, label=y_train)
valid_data = xgb.DMatrix(X_valid, label=y_valid)
test_data = xgb.DMatrix(temp_test)

model = xgb.train(param, train_data, evals=[(train_data, 'train'), (valid_data, 'valid')], num_boost_round = 10000, early_stopping_rounds=200, verbose_eval=25)
predict = model.predict(test_data)
test['Attrition']=predict
print(predict)

[01:30:49] WARNING: /Users/du/Downloads/xgboost/src/learner.cc:576: 
Parameters: { "boosting_type", "subsample_freq" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-auc:0.86523	valid-auc:0.72940
[25]	train-auc:0.95611	valid-auc:0.75483
[50]	train-auc:0.97197	valid-auc:0.76383
[75]	train-auc:0.98248	valid-auc:0.77322
[100]	train-auc:0.98972	valid-auc:0.77988
[125]	train-auc:0.99335	valid-auc:0.78140
[150]	train-auc:0.99595	valid-auc:0.78773
[175]	train-auc:0.99756	valid-auc:0.79035
[200]	train-auc:0.99843	valid-auc:0.79636
[225]	train-auc:0.99902	valid-auc:0.80203
[250]	train-auc:0.99952	valid-auc:0.80443
[275]	train-auc:0.99974	valid-auc:0.80738
[300]	train-auc:0.99987	valid-auc:0.80934
[325]	train-auc:0.99992	valid-auc:0.81043
[350]	train-auc:0.99995	valid-auc:0.812

In [22]:
# 转化为二分类输出
#test['Attrition']=test['Attrition'].map(lambda x:1 if x>=0.5 else 0)
# test[['Attrition']].to_csv('submit_xgb.csv')

In [23]:
### Attrition LGB

In [24]:
import pandas as pd

In [25]:
train=pd.read_csv(path + 'BI/train.csv',index_col=0)
test=pd.read_csv(path + 'BI/test.csv',index_col=0)

In [26]:
#print(train['Attrition'].value_counts())
# 处理Attrition字段
train['Attrition']=train['Attrition'].map(lambda x:1 if x=='Yes' else 0)
from sklearn.preprocessing import LabelEncoder
# 查看数据是否有空值
#print(train.isna().sum())

In [27]:
# 去掉没用的列 员工号码，标准工时（=80）
train = train.drop(['EmployeeNumber', 'StandardHours'], axis=1)
test = test.drop(['EmployeeNumber', 'StandardHours'], axis=1)

In [28]:
# 对于分类特征进行特征值编码
attr=['Age','BusinessTravel','Department','Education','EducationField','Gender','JobRole','MaritalStatus','Over18','OverTime']
lbe_list=[]
for feature in attr:
    lbe=LabelEncoder()
    train[feature]=lbe.fit_transform(train[feature])
    test[feature]=lbe.transform(test[feature])
    lbe_list.append(lbe)
#train.to_csv('temp.csv')
#print(train)

In [29]:
import lightgbm as lgb
from sklearn.model_selection import train_test_split

In [30]:
# param = {
#     'num_leaves':41,
#     'boosting_type': 'gbdt',
#     'objective':'binary',
#     'max_depth':15,
#     'learning_rate':0.001,
#     'metric':'binary_logloss'}
param = {'boosting_type':'gbdt',
                         'objective' : 'binary', #
                         #'metric' : 'binary_logloss',
                         'metric' : 'auc',
#                          'metric' : 'self_metric',
                         'learning_rate' : 0.01,
                         'max_depth' : 15,
                         'feature_fraction':0.8,
                         'bagging_fraction': 0.9,
                         'bagging_freq': 8,
                         'lambda_l1': 0.6,
                         'lambda_l2': 0,
#                          'scale_pos_weight':k,
#                         'is_unbalance':True
        }

In [31]:
X_train, X_valid, y_train, y_valid = train_test_split(train.drop('Attrition',axis=1), train['Attrition'], test_size=0.2, random_state=42)
train_data = lgb.Dataset(X_train, label=y_train)
valid_data = lgb.Dataset(X_valid, label=y_valid)


In [32]:
model = lgb.train(param,train_data,valid_sets=[train_data,valid_data],num_boost_round = 10000 ,early_stopping_rounds=200,verbose_eval=25, categorical_feature=attr)
predict=model.predict(test)
#print(predict)
test['Attrition']=predict
test['Attrition']=test['Attrition'].map(lambda x:1 if x>=0.5 else 0)
#test['Attrition']=predict
# test[['Attrition']].to_csv('submit_lgb.csv')

[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] Number of positive: 139, number of negative: 801
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000775 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1138
[LightGBM] [Info] Number of data points in the train set: 940, number of used features: 30
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.147872 -> initscore=-1.751387
[LightGBM] [Info] Start training from score -1.751387
[LightGBM] [Warning] No further splits with positive ga

In [33]:
### Attrition LR

In [34]:
import pandas as pd
from sklearn.feature_extraction import DictVectorizer

In [35]:
train=pd.read_csv(path + 'BI/train.csv',index_col=0)
test=pd.read_csv(path + 'BI/test.csv',index_col=0)

#print(train['Attrition'].value_counts())


In [36]:
# 处理Attrition字段
train['Attrition']=train['Attrition'].map(lambda x:1 if x=='Yes' else 0)
from sklearn.preprocessing import LabelEncoder
# 查看数据是否有空值
#print(train.isna().sum())

In [37]:
# 去掉没用的列 员工号码，标准工时（=80）
train = train.drop(['EmployeeNumber', 'StandardHours'], axis=1)
test = test.drop(['EmployeeNumber', 'StandardHours'], axis=1)

In [38]:
# 对于分类特征进行特征值编码
attr=['Age','BusinessTravel','Department','Education','EducationField','Gender','JobRole','MaritalStatus','Over18','OverTime']
lbe_list=[]
for feature in attr:
    lbe=LabelEncoder()
    train[feature]=lbe.fit_transform(train[feature])
    test[feature]=lbe.transform(test[feature])
    lbe_list.append(lbe)
#print(train)
# train.to_csv('train_label_encoder.csv')


In [39]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [40]:
X_train, X_valid, y_train, y_valid = train_test_split(train.drop('Attrition',axis=1), train['Attrition'], test_size=0.2, random_state=42)

In [41]:
model = LogisticRegression(max_iter=100, 
                           verbose=True, 
                           random_state=33,
                           tol=1e-4
                          )

In [42]:
model.fit(X_train, y_train)
predict = model.predict_proba(test)[:, 1]
test['Attrition']=predict

print(test['Attrition'])
# test[['Attrition']].to_csv('submit_lr.csv')
print('submit_lr.csv saved')
# 转化为二分类输出
#test['Attrition']=test['Attrition'].map(lambda x:1 if x>=0.5 else 0)
#test[['Attrition']].to_csv('submit_lr.csv')

RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =           33     M =           10

At X0         0 variables are exactly at the bounds

At iterate    0    f=  6.51558D+02    |proj g|=  4.65631D+06

At iterate   50    f=  3.67210D+02    |proj g|=  2.63990D+04

At iterate  100    f=  3.56557D+02    |proj g|=  1.60998D+04

           * * *

Tit   = total number of iterations
Tnf   = total number of function evaluations
Tnint = total number of segments explored during Cauchy searches
Skip  = number of BFGS updates skipped
Nact  = number of active bounds at final generalized Cauchy point
Projg = norm of the final projected gradient
F     = final function value

           * * *

   N    Tit     Tnf  Tnint  Skip  Nact     Projg        F
   33    100    112      1     0     0   1.610D+04   3.566D+02
  F =   356.55682351557238     

STOP: TOTAL NO. of ITERATIONS REACHED LIMIT                 
user_id
442     0.107292
1091    0.328317
981     0.240440
785     0.0

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
 This problem is unconstrained.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s finished


In [43]:
## Attrition NGBoost

In [44]:
import pandas as pd

In [45]:
train=pd.read_csv(path + 'BI/train.csv',index_col=0)
test=pd.read_csv(path + 'BI/test.csv',index_col=0)

In [46]:
#print(train['Attrition'].value_counts())
# 处理Attrition字段
train['Attrition']=train['Attrition'].map(lambda x:1 if x=='Yes' else 0)

In [47]:
from sklearn.preprocessing import LabelEncoder
# 查看数据是否有空值
#print(train.isna().sum())


In [48]:
# 去掉没用的列 员工号码，标准工时（=80）
train = train.drop(['EmployeeNumber', 'StandardHours'], axis=1)
test = test.drop(['EmployeeNumber', 'StandardHours'], axis=1)


In [49]:
# 对于分类特征进行特征值编码
attr=['Age','BusinessTravel','Department','Education','EducationField','Gender','JobRole','MaritalStatus','Over18','OverTime']
lbe_list=[]
for feature in attr:
    lbe=LabelEncoder()
    train[feature]=lbe.fit_transform(train[feature])
    test[feature]=lbe.transform(test[feature])
    lbe_list.append(lbe)
#print(train)

In [50]:
import ngboost as ng
import numpy as np
from sklearn.model_selection import train_test_split

In [51]:
X_train, X_valid, y_train, y_valid = train_test_split(train.drop('Attrition',axis=1), train['Attrition'], test_size=0.2, random_state=42)

In [52]:
model = ng.NGBClassifier(n_estimators=1000, 
                              learning_rate=0.01, 
                              verbose=True, 
                              verbose_eval=50
                             )


model.fit(X_train, y_train)


[iter 0] loss=0.4190 val_loss=0.0000 scale=1.0000 norm=2.0000
[iter 50] loss=0.3427 val_loss=0.0000 scale=1.0000 norm=1.7212
[iter 100] loss=0.3005 val_loss=0.0000 scale=1.0000 norm=1.6308
[iter 150] loss=0.2782 val_loss=0.0000 scale=1.0000 norm=1.5989
[iter 200] loss=0.2683 val_loss=0.0000 scale=1.0000 norm=1.5975
[iter 250] loss=0.2606 val_loss=0.0000 scale=1.0000 norm=1.5968
[iter 300] loss=0.2531 val_loss=0.0000 scale=1.0000 norm=1.5853
[iter 350] loss=0.2493 val_loss=0.0000 scale=0.5000 norm=0.7903
[iter 400] loss=0.2464 val_loss=0.0000 scale=1.0000 norm=1.5743
[iter 450] loss=0.2439 val_loss=0.0000 scale=0.5000 norm=0.7862
[iter 500] loss=0.2421 val_loss=0.0000 scale=0.0312 norm=0.0491
[iter 550] loss=0.2408 val_loss=0.0000 scale=1.0000 norm=1.5703
[iter 600] loss=0.2405 val_loss=0.0000 scale=0.0002 norm=0.0004
[iter 650] loss=0.2402 val_loss=0.0000 scale=0.0002 norm=0.0004
[iter 700] loss=0.2402 val_loss=0.0000 scale=0.0002 norm=0.0004
[iter 750] loss=0.2402 val_loss=0.0000 scal

NGBClassifier(n_estimators=1000,
              random_state=RandomState(MT19937) at 0x11513C340,
              verbose_eval=50)

In [53]:
#predict = model.predict(test)
predict = model.predict_proba(test)[:, 1]
#print(predict)
test['Attrition']=predict
## 转化为二分类输出
test['Attrition']=test['Attrition'].map(lambda x:1 if x>=0.5 else 0)
# test[['Attrition']].to_csv('submit_ngb.csv')


In [54]:
### Attrition SVC

In [55]:
import pandas as pd

In [56]:
train=pd.read_csv(path + 'BI/train.csv',index_col=0)
test=test1=pd.read_csv(path + 'BI/test.csv',index_col=0)

In [57]:
#print(train['Attrition'].value_counts())
# 处理Attrition字段
train['Attrition']=train['Attrition'].map(lambda x:1 if x=='Yes' else 0)

In [58]:
from sklearn.preprocessing import LabelEncoder
# 查看数据是否有空值
#print(train.isna().sum())

In [59]:
# 去掉没用的列 员工号码，标准工时（=80）
train = train.drop(['EmployeeNumber', 'StandardHours'], axis=1)
test = test.drop(['EmployeeNumber', 'StandardHours'], axis=1)


In [60]:
# 对于分类特征进行特征值编码
attr=['Age','BusinessTravel','Department','Education','EducationField','Gender','JobRole','MaritalStatus','Over18','OverTime']
lbe_list=[]
for feature in attr:
    lbe=LabelEncoder()
    train[feature]=lbe.fit_transform(train[feature])
    test[feature]=lbe.transform(test[feature])
    lbe_list.append(lbe)
#print(train)

In [61]:
from sklearn.svm import SVC, LinearSVC
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

In [62]:
X_train, X_valid, y_train, y_valid = train_test_split(train.drop('Attrition',axis=1), train['Attrition'], test_size=0.2, random_state=42)


mms = MinMaxScaler(feature_range=(0, 1))
X_train = mms.fit_transform(X_train)
X_valid = mms.fit_transform(X_valid)
test = mms.fit_transform(test)

In [63]:
model = SVC(kernel='rbf', 
			gamma="auto",
			max_iter=1000,
			random_state=33,
			verbose=True,
			tol=1e-5,
			cache_size=50000
		   )
#print(X_train)
#print(y_train)
#print(sum(y_train))

model = LinearSVC(
			max_iter=1000,
			random_state=33,
			verbose=True,
		   )
model.fit(X_train, y_train)
predict = model.predict(test)
print(predict)
#print(test)
#predict = model.predict_proba(test)[:, 1]
test1['Attrition']=predict

[LibLinear]...............................*[0 0 0 0 1 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0]
optimization finished, #iter = 315
Objective value = -359.033106
nSV = 614



In [64]:
# 转化为二分类输出
#test['Attrition']=test['Attrition'].map(lambda x:1 if x>=0.5 else 0)
# test1[['Attrition']].to_csv('submit_svc.csv')

In [65]:
### Attrition XGBoost

In [66]:
import pandas as pd

In [67]:
train=pd.read_csv(path + 'BI/train.csv',index_col=0)
test=pd.read_csv(path + 'BI/test.csv',index_col=0)

In [68]:
#print(train['Attrition'].value_counts())
# 处理Attrition字段
train['Attrition']=train['Attrition'].map(lambda x:1 if x=='Yes' else 0)

In [69]:
from sklearn.preprocessing import LabelEncoder
# 查看数据是否有空值
#print(train.isna().sum())

In [70]:
# 去掉没用的列 员工号码，标准工时（=80）
train = train.drop(['EmployeeNumber', 'StandardHours'], axis=1)
test = test.drop(['EmployeeNumber', 'StandardHours'], axis=1)

In [71]:
# 对于分类特征进行特征值编码
attr=['Age','BusinessTravel','Department','Education','EducationField','Gender','JobRole','MaritalStatus','Over18','OverTime']
lbe_list=[]
for feature in attr:
    lbe=LabelEncoder()
    train[feature]=lbe.fit_transform(train[feature])
    test[feature]=lbe.transform(test[feature])
    lbe_list.append(lbe)
#train.to_csv('temp.csv')
#print(train)

In [72]:
import xgboost as xgb
from sklearn.model_selection import train_test_split

In [73]:
param = {'boosting_type':'gbdt',
                         'objective' : 'binary:logistic', #
                         'eval_metric' : 'auc',
                         'eta' : 0.01,
                         'max_depth' : 15,
                         'colsample_bytree':0.8,
                         'subsample': 0.9,
                         'subsample_freq': 8,
                         'alpha': 0.6,
                         'lambda': 0,
}

In [74]:
X_train, X_valid, y_train, y_valid = train_test_split(train.drop('Attrition',axis=1), train['Attrition'], test_size=0.2, random_state=42)

In [75]:
train_data = xgb.DMatrix(X_train, label=y_train)
valid_data = xgb.DMatrix(X_valid, label=y_valid)
test_data = xgb.DMatrix(test)

model = xgb.train(param, train_data, evals=[(train_data, 'train'), (valid_data, 'valid')], num_boost_round = 10000, early_stopping_rounds=200, verbose_eval=25)
predict = model.predict(test_data)
test['Attrition']=predict
print(predict)

[01:40:57] WARNING: /Users/du/Downloads/xgboost/src/learner.cc:576: 
Parameters: { "boosting_type", "subsample_freq" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[0]	train-auc:0.81943	valid-auc:0.57257
[25]	train-auc:0.96269	valid-auc:0.76503
[50]	train-auc:0.97533	valid-auc:0.77829
[75]	train-auc:0.98602	valid-auc:0.78020
[100]	train-auc:0.99019	valid-auc:0.78129
[125]	train-auc:0.99288	valid-auc:0.78402
[150]	train-auc:0.99539	valid-auc:0.79374
[175]	train-auc:0.99724	valid-auc:0.79385
[200]	train-auc:0.99857	valid-auc:0.80181
[225]	train-auc:0.99903	valid-auc:0.80498
[250]	train-auc:0.99934	valid-auc:0.80738
[275]	train-auc:0.99970	valid-auc:0.80781
[300]	train-auc:0.99986	valid-auc:0.80956
[325]	train-auc:0.99992	valid-auc:0.81098
[350]	train-auc:0.99994	valid-auc:0.811

In [76]:
# 转化为二分类输出
#test['Attrition']=test['Attrition'].map(lambda x:1 if x>=0.5 else 0)
# test[['Attrition']].to_csv('submit_xgb.csv')

In [130]:
## Manage